# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jul 26 2022 11:19AM,242744,19,ACG-2102490720,ACG North America LLC,Released
1,Jul 26 2022 11:07AM,242743,10,0085907548,ISDIN Corporation,Released
2,Jul 26 2022 11:07AM,242743,10,0085907549,ISDIN Corporation,Released
3,Jul 26 2022 11:07AM,242743,10,0085907564,ISDIN Corporation,Released
4,Jul 26 2022 11:07AM,242743,10,0085907662,ISDIN Corporation,Released
5,Jul 26 2022 11:07AM,242743,10,0085907663,ISDIN Corporation,Released
6,Jul 26 2022 11:07AM,242743,10,0085908009,ISDIN Corporation,Released
7,Jul 26 2022 11:06AM,242742,10,SONSB0001880,"Nextsource Biotechnology, LLC",Released
8,Jul 26 2022 11:01AM,242741,10,PRF-32877-BO,Bio-PRF,Released
9,Jul 26 2022 11:01AM,242741,10,PRF-33876,Bio-PRF,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
37,242740,Released,1
38,242741,Released,11
39,242742,Released,1
40,242743,Released,6
41,242744,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
242740,NaN,NaN,1.0
242741,NaN,NaN,11.0
242742,NaN,NaN,1.0
242743,NaN,NaN,6.0
242744,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
242581,0.0,0.0,1.0
242685,0.0,1.0,0.0
242696,45.0,8.0,0.0
242709,0.0,2.0,2.0
242710,0.0,8.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
242581,0,0,1
242685,0,1,0
242696,45,8,0
242709,0,2,2
242710,0,8,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,242581,0,0,1
1,242685,0,1,0
2,242696,45,8,0
3,242709,0,2,2
4,242710,0,8,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,242581,,,1
1,242685,,1,
2,242696,45,8,
3,242709,,2,2
4,242710,,8,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jul 26 2022 11:19AM,242744,19,ACG North America LLC
1,Jul 26 2022 11:07AM,242743,10,ISDIN Corporation
7,Jul 26 2022 11:06AM,242742,10,"Nextsource Biotechnology, LLC"
8,Jul 26 2022 11:01AM,242741,10,Bio-PRF
19,Jul 26 2022 10:59AM,242740,12,HealthCaps LLC
20,Jul 26 2022 10:52AM,242735,10,Eywa Pharma Inc.
35,Jul 26 2022 10:39AM,242739,19,ACG North America LLC
36,Jul 26 2022 10:24AM,242736,10,MedStone Pharma LLC
41,Jul 26 2022 10:20AM,242730,15,"Person & Covey, Inc."
50,Jul 26 2022 10:15AM,242731,15,"Virtus Pharmaceuticals, LLC"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jul 26 2022 11:19AM,242744,19,ACG North America LLC,,,1
1,Jul 26 2022 11:07AM,242743,10,ISDIN Corporation,,,6
2,Jul 26 2022 11:06AM,242742,10,"Nextsource Biotechnology, LLC",,,1
3,Jul 26 2022 11:01AM,242741,10,Bio-PRF,,,11
4,Jul 26 2022 10:59AM,242740,12,HealthCaps LLC,,,1
5,Jul 26 2022 10:52AM,242735,10,Eywa Pharma Inc.,,,15
6,Jul 26 2022 10:39AM,242739,19,ACG North America LLC,,,1
7,Jul 26 2022 10:24AM,242736,10,MedStone Pharma LLC,,,5
8,Jul 26 2022 10:20AM,242730,15,"Person & Covey, Inc.",,,9
9,Jul 26 2022 10:15AM,242731,15,"Virtus Pharmaceuticals, LLC",,27,2


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 26 2022 11:19AM,242744,19,ACG North America LLC,1,,
1,Jul 26 2022 11:07AM,242743,10,ISDIN Corporation,6,,
2,Jul 26 2022 11:06AM,242742,10,"Nextsource Biotechnology, LLC",1,,
3,Jul 26 2022 11:01AM,242741,10,Bio-PRF,11,,
4,Jul 26 2022 10:59AM,242740,12,HealthCaps LLC,1,,
5,Jul 26 2022 10:52AM,242735,10,Eywa Pharma Inc.,15,,
6,Jul 26 2022 10:39AM,242739,19,ACG North America LLC,1,,
7,Jul 26 2022 10:24AM,242736,10,MedStone Pharma LLC,5,,
8,Jul 26 2022 10:20AM,242730,15,"Person & Covey, Inc.",9,,
9,Jul 26 2022 10:15AM,242731,15,"Virtus Pharmaceuticals, LLC",2,27,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 26 2022 11:19AM,242744,19,ACG North America LLC,1,,
1,Jul 26 2022 11:07AM,242743,10,ISDIN Corporation,6,,
2,Jul 26 2022 11:06AM,242742,10,"Nextsource Biotechnology, LLC",1,,
3,Jul 26 2022 11:01AM,242741,10,Bio-PRF,11,,
4,Jul 26 2022 10:59AM,242740,12,HealthCaps LLC,1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 26 2022 11:19AM,242744,19,ACG North America LLC,1.0,NaN,NaN
1,Jul 26 2022 11:07AM,242743,10,ISDIN Corporation,6.0,NaN,NaN
2,Jul 26 2022 11:06AM,242742,10,"Nextsource Biotechnology, LLC",1.0,NaN,NaN
3,Jul 26 2022 11:01AM,242741,10,Bio-PRF,11.0,NaN,NaN
4,Jul 26 2022 10:59AM,242740,12,HealthCaps LLC,1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 26 2022 11:19AM,242744,19,ACG North America LLC,1.0,0.0,0.0
1,Jul 26 2022 11:07AM,242743,10,ISDIN Corporation,6.0,0.0,0.0
2,Jul 26 2022 11:06AM,242742,10,"Nextsource Biotechnology, LLC",1.0,0.0,0.0
3,Jul 26 2022 11:01AM,242741,10,Bio-PRF,11.0,0.0,0.0
4,Jul 26 2022 10:59AM,242740,12,HealthCaps LLC,1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,2912572,65.0,16.0,45.0
12,242740,1.0,0.0,0.0
15,1456383,58.0,43.0,0.0
16,485442,6.0,22.0,0.0
18,242685,0.0,1.0,0.0
19,485483,2.0,0.0,0.0
20,1699012,113.0,69.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,2912572,65.0,16.0,45.0
1,12,242740,1.0,0.0,0.0
2,15,1456383,58.0,43.0,0.0
3,16,485442,6.0,22.0,0.0
4,18,242685,0.0,1.0,0.0
5,19,485483,2.0,0.0,0.0
6,20,1699012,113.0,69.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,65.0,16.0,45.0
1,12,1.0,0.0,0.0
2,15,58.0,43.0,0.0
3,16,6.0,22.0,0.0
4,18,0.0,1.0,0.0
5,19,2.0,0.0,0.0
6,20,113.0,69.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,65.0
1,12,Released,1.0
2,15,Released,58.0
3,16,Released,6.0
4,18,Released,0.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,18,19,20
Status,,,,,,,
Completed,45.0,0.0,0.0,0.0,0.0,0.0,0.0
Executing,16.0,0.0,43.0,22.0,1.0,0.0,69.0
Released,65.0,1.0,58.0,6.0,0.0,2.0,113.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,18,19,20
0,Completed,45.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Executing,16.0,0.0,43.0,22.0,1.0,0.0,69.0
2,Released,65.0,1.0,58.0,6.0,0.0,2.0,113.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,18,19,20
0,Completed,45.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Executing,16.0,0.0,43.0,22.0,1.0,0.0,69.0
2,Released,65.0,1.0,58.0,6.0,0.0,2.0,113.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()